<a target="_blank" href="https://colab.research.google.com/github/VectorInstitute/fed-rag/blob/main/docs/notebooks/rag_benchmarking_hf_mmlu.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

_(NOTE: if running on Colab, you will need to supply a WandB API Key in addition to your HFToken. Also, you'll need to change the runtime to a T4.)_

In [1]:
!uv pip install bitsandbytes -q

In [1]:
!uv pip install docker -q

# Basic Federated Fine-tuning of RAG Systems

## Knowledge Store

In [1]:
import docker
import os
import time

client = docker.from_env()
image_name = "vectorinstitute/qdrant-atlas-dec-wiki-2021:latest"

# first see if we need to pull the docker image
try:
    client.images.get(image_name)
    print(f"Image '{image_name}' already exists locally")
except docker.errors.ImageNotFound:
    print(f"Image '{image_name}' not found locally. Pulling...")
    # Pull with progress information
    for line in client.api.pull(image_name, stream=True, decode=True):
        if "progress" in line:
            print(f"\r{line['status']}: {line['progress']}", end="")
        elif "status" in line:
            print(f"\r{line['status']}", end="")
    print("\nPull complete!")

# run the Qdrant container
container = client.containers.run(
    "vectorinstitute/qdrant-atlas-dec-wiki-2021:latest",
    detach=True,  # -d flag
    name="tiny-wiki-dec2021-ks",  # --name
    ports={"6333/tcp": 6333, "6334/tcp": 6334},  # -p 6333:6333  # -p 6334:6334
    volumes={
        "qdrant_data": {  # -v qdrant_data:/qdrant_storage
            "bind": "/qdrant_storage",
            "mode": "rw",
        }
    },
    environment={"SAMPLE_SIZE": "tiny"},  # -e SAMPLE_SIZE=tiny
    device_requests=[
        docker.types.DeviceRequest(
            count=-1, capabilities=[["gpu"]]
        )  # --gpus all
    ],
    remove=False,  # Don't auto-remove when stopped
)

print(f"Container started with ID: {container.id}")

# wait a moment for the container to initialize
time.sleep(3)

# Check container status
container.reload()  # Refresh container data
print(f"Container status: {container.status}")
print(f"Container logs:")
print(container.logs().decode("utf-8"))

Image 'vectorinstitute/qdrant-atlas-dec-wiki-2021:latest' already exists locally
Container started with ID: 3bbf28156093168dc6ad130fa9833c0a23804841b34897c91a476596c5daef20
Container status: running
Container logs:
Starting Qdrant Atlas Knowledge Store container
Running database initialization check...
Using tiny sample mode...
Creating tiny sample file for testing...
Using tiny sample file: tiny-sample.jsonl
Verifying sample file creation...
✅ Sample file successfully created at: /app/data/atlas/enwiki-dec2021/tiny-sample.jsonl
File details:
-rw-r--r-- 1 root root 6785 Jun  4 15:06 /app/data/atlas/enwiki-dec2021/tiny-sample.jsonl
File content (first 3 lines):
{"id": "140", "title": "History of marine biology", "section": "James Cook", "text": " James Cook is well known for his voyages of exploration for the British Navy in which he mapped out a significant amount of the world's uncharted waters. Cook's explorations took him around the world twice and led to countless descriptions of p

In [4]:
container.health

'starting'

In [2]:
print(f"Container status: {container.status}")

Container status: running


In [5]:
from fed_rag.knowledge_stores import QdrantKnowledgeStore

In [8]:
store = QdrantKnowledgeStore(
    collection_name="nthakur.dragon-plus-context-encoder"
)

In [9]:
store.count

ResponseHandlingException: [Errno 111] Connection refused

In [2]:
import time

GIST_URL = f"https://gist.githubusercontent.com/nerdai/33e8445ab8b96783f34a7e0464e0b0f0/raw?fresh={int(time.time())}"

In [3]:
import requests

response = requests.get(GIST_URL)
rag_code = response.text

In [4]:
from IPython.display import Code, display

display(Code(rag_code, language="python"))

import torch
from transformers.generation.utils import GenerationConfig
from transformers.utils.quantization_config import BitsAndBytesConfig
from datasets import Dataset
from typing import Literal

from fed_rag import RAGSystem, RAGConfig
from fed_rag.knowledge_stores import InMemoryKnowledgeStore
from fed_rag.retrievers import (
    HFSentenceTransformerRetriever,
)
from fed_rag.generators import HFPeftModelGenerator
from fed_rag.fl_tasks.huggingface import (
    HuggingFaceFlowerClient,
    HuggingFaceFlowerServer,
)
from fed_rag.data_structures import KnowledgeNode, NodeType
from fed_rag.trainers.huggingface.ralt import HuggingFaceTrainerForRALT
from fed_rag.trainer_managers.huggingface import HuggingFaceRAGTrainerManager

GRPC_MAX_MESSAGE_LENGTH = int(512 * 1024 * 1024 * 3.75)
PEFT_MODEL_NAME = "Styxxxx/llama2_7b_lora-quac"
BASE_MODEL_NAME = "meta-llama/Llama-2-7b-hf"
TEXT_CHUNKS = [ # a small sample from the Dec 2021 Wikipedia dump for knowledge store
    {
        "id": "140",
        "title": "History of marine biology",
        "section": "James Cook",
        "text": " James Cook is well known for his voyages of exploration for the British Navy in which he mapped out a significant amount of the world's uncharted waters. Cook's explorations took him around the world twice and led to countless descriptions of previously unknown plants and animals. Cook's explorations influenced many others and led to a number of scientists examining marine life more closely. Among those influenced was Charles Darwin who went on to make many contributions of his own. ",
    },
    {
        "id": "141",
        "title": "History of marine biology",
        "section": "Charles Darwin",
        "text": " Charles Darwin, best known for his theory of evolution, made many significant contributions to the early study of marine biology. He spent much of his time from 1831 to 1836 on the voyage of HMS Beagle collecting and studying specimens from a variety of marine organisms. It was also on this expedition where Darwin began to study coral reefs and their formation. He came up with the theory that the overall growth of corals is a balance between the growth of corals upward and the sinking of the sea floor. He then came up with the idea that wherever coral atolls would be found, the central island where the coral had started to grow would be gradually subsiding",
    },
    {
        "id": "142",
        "title": "History of marine biology",
        "section": "Charles Wyville Thomson",
        "text": " Another influential expedition was the voyage of HMS Challenger from 1872 to 1876, organized and later led by Charles Wyville Thomson. It was the first expedition purely devoted to marine science. The expedition collected and analyzed thousands of marine specimens, laying the foundation for present knowledge about life near the deep-sea floor. The findings from the expedition were a summary of the known natural, physical and chemical ocean science to that time.",
    },
    {
        "id": "143",
        "title": "History of marine biology",
        "section": "Later exploration",
        "text": " This era of marine exploration came to a close with the first and second round-the-world voyages of the Danish Galathea expeditions and Atlantic voyages by the USS Albatross, the first research vessel purpose built for marine research. These voyages further cleared the way for modern marine biology by building a base of knowledge about marine biology. This was followed by the progressive development of more advanced technologies which began to allow more extensive explorations of ocean depths that were once thought too deep to sustain life.",
    },
    {
        "id": "144",
        "title": "History of marine biology",
        "section": "Marine biology labs",
        "text": " In the 1960s and 1970s, ecological research into the life of the ocean was undertaken at institutions set up specifically to study marine biology. Notable was the Woods Hol

In [5]:
# load the vars from the gist
setup_only_code = rag_code.replace(
    'if __name__ == "__main__":', "if False:  # Disabled for notebook"
)

before_exec = set(globals().keys())
exec(setup_only_code)

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
knowledge_store.count

13

### Centralized Training

In [ ]:
result = manager.train()

In [ ]:
result.loss

In [ ]:
# After exec - find what was added
after_exec = set(globals().keys())
new_vars = after_exec - before_exec

# Delete everything that was added
for var_name in new_vars:
    del globals()[var_name]

print(f"🗑️ Deleted {len(new_vars)} variables: {new_vars}")

In [ ]:
import torch
import gc

torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# More aggressive cleanup
torch.cuda.synchronize()
torch.cuda.empty_cache()
gc.collect()

### Federated Learning

In [ ]:
# write gist to script
with open("rag_federated_learning.py", "w") as f:
    f.write(rag_code)

In [ ]:
server_command = "python rag_federated_learning.py --component server"
client_command = "python rag_federated_learning.py --component {client_name}"

In [ ]:
from fed_rag.utils.cookbook import ProcessMonitor

monitor = ProcessMonitor()

In [ ]:
monitor.start_process("server", server_command)

In [ ]:
monitor.start_process(
    "client_1", client_command.format(client_name="client_1")
)
monitor.start_process(
    "client_2", client_command.format(client_name="client_2")
)

In [ ]:
print(monitor.get_logs("server"))

In [ ]:
print(monitor.get_logs("client_1"))

In [ ]:
print(monitor.get_logs("client_2"))

In [ ]:
# monitor.stop_all()

In [ ]:
monitor.list_processes()

### Cleanup

In [9]:
# stop and remove container
container.stop()
container.remove()